In [1]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive
from VC_collections.AuthorityFiles import Authority_instance
from VC_collections.authorities import check_lang
import gspread
import pandas as pd


In [ ]:
cred = Collection.get_google_drive_credentials()
client = gspread.authorize(cred)
len(client.list_spreadsheet_files())

In [ ]:
collection = Collection.retrieve_collection()

In [ ]:
check_lang(collection.df_final_data.loc[collection.df_final_data["תיאור"]!='']["תיאור"].tolist()[0])

In [ ]:
check_lang("asdsdf")

In [ ]:
from VC_collections.AuthorityFiles import create_df_from_gs
spreadsheet = client.open_by_url(
            "https://docs.google.com/spreadsheets/d/1736sL9unbiOMbcrIYgSkCSvhU2-LCthSLVtYLPSpZ98"
        )


In [ ]:
 Authority_instance.df_countries_ara.to_dict()["MARC"]["سويسرا"]

In [ ]:
Authority_instance.language_mapping_dict_ara

# Country of publication

In [ ]:
df_country, df_country_cols = create_df_from_gs(spreadsheet, "מדינת פרסום")

In [ ]:
df_country.head()

In [ ]:
Authority_instance.df_countries.set_index("מדינת פרסום בערבית").to_dict()["MARC"]

In [ ]:
Authority_instance.countries_mapping_dict_ara

In [ ]:
{k: v for k, v in Authority_instance.countries_mapping_dict_ara.items() if v}

# Subjects

In [ ]:
df_subjects, df_subjects_cols = create_df_from_gs(spreadsheet, "נושאים")
df_subjects.head()

In [ ]:
df_subjects_mapper = df_subjects.set_index("נושא עברית").to_dict()["650 7"]
for key, value in df_subjects_mapper.items():
    print(f'key: {key}, value: {value}')

In [ ]:
type(df_subjects)

In [ ]:
collection= Collection.Collection("Alma", "Design", "PShHi")

In [ ]:
collection.df_final_data["בעלים נוכחי"].tolist()[0]

In [ ]:
values_list = list(set(filter(None,";".join(collection.df_final_data["סוג חומר"].tolist()).split(";"))))

In [ ]:
values_list.append("עבודה אקדמית")

In [ ]:
from VC_collections.authorities import convert_dict, fix_original

fix_original()

auth_list = convert_dict(Authority_instance.arch_mat_search_dict)
auth_list

In [ ]:
values_not_found = []

for value in values_list:
    if value in auth_list.keys():
        print(f'original {value} is in Authorities')
    else:
        print(f'!!!!!!original {value} not in Authorities') 
        values_not_found.append(value)

In [ ]:
auth_list['עבודה אקדמית']

In [ ]:
len(list(filter(None,";".join(collection.df_final_data["סוג חומר"].tolist()).split(";"))))


In [ ]:
with open(r'Data\VIS_907.json') as json_file:
    rosetta_dict = json.load(json_file)

In [ ]:
rosetta_dict

In [ ]:
Collection.collection_field_mapper.keys()

# Archival Material

In [ ]:
from VC_collections.AuthorityFiles import Authority_instance
import pprint
from collections import defaultdict

In [ ]:
Authority_instance.df_arch_mat_search["all_terms"]

In [ ]:
def convert_dict(new_val2errs):
    d = defaultdict(list)
    for nv, errs in new_val2errs.items():
        if type(errs) == str:
            errs = errs.split(";")
        for err in errs:
            if err != "":
                d[err].append(nv)
    return d

pprint.pprint(convert_dict(Authority_instance.arch_mat_search_dict))

In [ ]:
pprint.pprint(Authority_instance.arch_mat_search_dict)

# Media Format

In [ ]:
Authority_instance.df_media_format_auth[Authority_instance.df_media_format_auth["MEDIA_FORMAT"].str.contains("עץ")]

# Privacy

In [ ]:
privacy_mapping_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].values,
                                 index=Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
privacy_mapping_dict

In [ ]:
search_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].apply(lambda x: x.split(";")), 
                                                               index = Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
new_search_dict = {}

for key, value in search_dict.items():
    if len(value)>1:
        for item in value:
            new_search_dict[item] = key
    else:
        new_search_dict[value[0]] = key

In [ ]:
new_search_dict.keys()

In [ ]:
from VC_collections.authorities import convert_dict
convert_dict(new_search_dict)

In [ ]:
for key, value in convert_dict(new_search_dict).items():
    print(f'key:{key}\nvalue:{value}\n')

In [ ]:
for key, value in Authority_instance.arch_mat_search_dict.items():
    print(f'key:{key}\nvalue:{value}\n')


# Subjects

In [ ]:
# Authority_instance.df_subject_mapper

subject_mapper = {k: v for k, v in Authority_instance.df_subject_mapper.items() if v}

# split creators

In [ ]:
from VC_collections.AuthorityFiles import Authority_instance
from VC_collections.authorities import find_role, find_name
import pandas as pd

In [ ]:
Authority_instance.df_creator_pers_role.head()

In [ ]:
Authority_instance.df_creator_pers_role["CREATORS_PERS_ROLE_ARA"]

In [ ]:
Authority_instance.df_creator_corps_role.head()

In [ ]:
df = pd.ExcelFile(r'Data/PYaVa_Creators.xlsx').parse("Sheet1")
df = df.set_index("סימול")

In [ ]:
df["יוצר ראשי - איש"] = ''
df["סוג יוצר ראשי - איש"] = ''
df["יוצר ראשי - מוסד"] = ''
df["סוג יוצר ראשי - מוסד"] = ''
df["יוצרים נוספים - איש"] = ''
df["יוצרים נוספים - מוסד"] = ''

In [ ]:

def create_first(creators):

    creator = creators.pop(0)
    
    return find_name(creator).strip(), find_role(creator).strip(), creators

In [ ]:
import numpy as np
roles_not_found = []

for index, row in df.iterrows():

    creators_pers = []
    creators_corps = []
    try:
        creators = row["יוצרים"].split(";")
    except:
        continue
        
    
    
    first_name, first_role, rest_creators = create_first(creators)
    
    if first_role in Authority_instance.roles_dict["pers_roles"]:
        df.loc[index, "יוצר ראשי - איש"] = first_name
        df.loc[index, "סוג יוצר ראשי - איש"] = first_role
    else:
        df.loc[index, "יוצר ראשי - מוסד"] = first_name
        df.loc[index, "סוג יוצר ראשי - מוסד"] = first_role
    
    for creator in rest_creators:
        print(index)
        role = find_role(creator).strip()
        if role in Authority_instance.roles_dict["pers_roles"]:
            print(f'role: {role} is pers')
            creators_pers.append(creator)
        elif role in Authority_instance.roles_dict["corps_roles"]:
            creators_corps.append(creator)
            print(f'role: {role} is corps')
        else:
            roles_not_found.append(creator)
            
    df.loc[index, "יוצרים נוספים - איש"] = ";".join(creators_pers)
    df.loc[index, "יוצרים נוספים - מוסד"] = ";".join(creators_corps)
    if len(roles_not_found) > 0:
        print(roles_not_found)
    

In [ ]:
roles_not_found

In [ ]:
df.to_excel(r'Data/PYaVa_creators_repair.xlsx')

In [ ]:
collection.df_final_data["כותרת ערבית"].empty

In [ ]:
collection.df_final_data.columns

In [ ]:
import pandas as pd

In [ ]:
d= {'ניסיון': ['', '', '']}

In [ ]:
test = pd.DataFrame(d)

In [ ]:

len(test[test["ניסיון"] != ''])

In [ ]:
"ArBe" in Authority_instance.df_credits.index

In [ ]:
Authority_instance.df_credits.loc["ArBe", "מיקום הפקדה עבור בעלים נוכחי"]

In [ ]:
 test_data_with_dup = {
            "UNITID": ["ABCD", "ABCD", "ASDF"]
        }

In [ ]:
df = pd.DataFrame(test_data_with_dup)
df

In [ ]:
from VC_collections.columns import dupCheck
import pandas as pd

In [ ]:
dupCheck(df, "UNITID")

In [ ]:

pd.concat(g for _, g in df.groupby("UNITID") if len(g) > 1)

In [ ]:
import numpy as np
df["test"] = ''

In [ ]:
df

In [ ]:
df["test"].dropna().empty

In [ ]:
df["test"] = df["test"].dropna()

In [ ]:
df

In [ ]:
df["test"].empty

In [ ]:
test_008 = "191104k19811981is#xx###############heb#d"

In [ ]:
test_008[:7]  + "19841984" + test_008[15:]

In [ ]:
from viapy import api

In [ ]:
from urllib.request import urlopen, quote
# if you're running python 3, replace the above with the following:
# from urllib.request import urlopen
# from urllib.parse import quote

def retrieve_viaf_search_results(search_index, search_term, auth_source):
    # url search template formatted to allow easy variable insertion
    search_url_template = 'http://viaf.org/viaf/search/viaf?query=local.{0}+all+{1}+and+local.sources+any+{2}&sortKeys=holdingscount&httpAccept=application/xml'
    
    # since we'll be inserting the three passed variables into the 
    # html search template, we need to make sure to "escape" any
    # special characters. VIAF also requires that they be enclosed in
    # quotes, which themselves have to be escaped. We can use urllib2's 
    # "quote" function for this:
    auth_source = quote(f'"{auth_source}"')
    search_term = quote(f'"{search_term}"')
    
    # build the full search url
    search_url = search_url_template.format(search_index, search_term, auth_source)
    
    # query the url with urlopen and store the returned xml
    response = urlopen(search_url).read()

In [ ]:
retrieve_viaf_search_results("persname", "צרפתי, רות")

# Arabic

In [ ]:
spreadsheet = client.open_by_url(
            "https://docs.google.com/spreadsheets/d/1736sL9unbiOMbcrIYgSkCSvhU2-LCthSLVtYLPSpZ98")

In [ ]:
from VC_collections.AuthorityFiles import Authority

In [ ]:
import alphabet_detector

In [ ]:
ad = alphabet_detector.AlphabetDetector()

In [ ]:
test_lang = "'اسرائيل;الاردن'"

In [ ]:
ad.detect_alphabet(test_lang)

In [ ]:
check_lang(test_lang)

In [ ]:
Authority_instance.df_creator_pers_role

In [ ]:
language_mapping_dict_ara = Authority_instance.df_languages.loc[Authority_instance.df_languages["שם שפה ערבית"]!=""].reset_index().set_index("שם שפה ערבית")[["קוד שפה"]].to_dict()['קוד שפה']

In [ ]:
Authority_instance.df_languages.head()

In [ ]:
Authority_instance.df_languages.loc[Authority_instance.df_languages.loc[Authority_instance.df_languages["שם שפה ערבית"]=="العبرية"].index[0], "קוד שפה"]

In [ ]:

test_ara_dict = Authority_instance.df_languages[["שם שפה ערבית", "קוד שפה"]].to_dict()
for k, v in language_mapping_dict_ara.items():
    print(f'k: {k}, v: {v}, language: {check_lang(k)}\n')


# test_ara_dict.values()

In [ ]:
# language_mapping_dict_ara["الانجليزية"]

In [ ]:
eng1 = "الإنجليزية"
eng2 = "الانجليزية"

for i in range(10):
    print(eng1[i], " | ", eng2[i], " | ", eng1[i]==eng2[i])

In [ ]:
Authority_instance.df_languages.to_dict()["קוד שפה"]["שלום"]

In [ ]:
{k: v for k, v in test_ara_dict.items() if k != ''}


In [ ]:
Authority_instance.language_mapping_dict

In [ ]:
Authority_instance.df_languages.to_dict()

## xml 

In [13]:
import xml.etree.ElementTree as ET
from pathlib import Path
import pymarc
from pymarc import XmlHandler

file_path = r"G:\My Drive\National_Library\Python\VC-Dance\DCSD\Digitization\ROS\DCSD_907.xml"

In [ ]:
tree = ET.parse(file_path)
root = tree.getroot()

In [14]:

records = pymarc.marcxml.parse_xml_to_array(file_path)

In [15]:
from collections import defaultdict

d = defaultdict(list)

for record in records:
    for field in record.get_fields("952"):
        print("$$"+ " ".join(str(field.as_marc(encoding="utf8"), "utf8").split()))

#         print(record["001"].value(), field.subfields)
#     print([x for x in record.get_fields("915").get_subfields()])
        
#     d[record["001"].value()] = ";".join([x.subfields() for x in record.get_fields("915")])



$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by Ruben Wengiel 20220214 dייצוא דוח נתונים
$$aIn copyright; Contract bProtected during author lifetime plus 70 years cNoam Solan by R

In [ ]:
for key, val in dict(d).items():
    print(f"key: {key}, val: {val}")

In [ ]:
dict(d)

In [16]:
def create_field_dict(file, tag):
    handler = XmlHandler()
    records = pymarc.marcxml.parse_xml_to_array(file)
    dd = defaultdict(lambda: defaultdict(str))
    for record in records:

        index = 1
        for field in record.get_fields(tag):
            key = f'{field.tag}{"".join(field.indicators)}_{index}'
            key = "".join([x for x in key if x != " "])
            value = ""
            for subfield in field.subfields:
                if len(subfield) == 1:
                    value += f"$${subfield}"
                else:
                    value += subfield
            dd[record["001"].value()][key] = value
            index += 1

    return dd

In [20]:
from xml.dom import minidom

# rosetta_file = minidom.parse(file_path)

create_field_dict(file_path, "952")

defaultdict(<function __main__.create_field_dict.<locals>.<lambda>()>,
            {'990049233600205171': defaultdict(str,
                         {'952_1': '$$aIn copyright; Contract$$bProtected during author lifetime plus 70 years$$cNoam Solan by Ruben Wengiel 20220214$$dייצוא דוח נתונים'}),
             '990049233610205171': defaultdict(str,
                         {'952_1': '$$aIn copyright; Contract$$bProtected during author lifetime plus 70 years$$cNoam Solan by Ruben Wengiel 20220214$$dייצוא דוח נתונים'}),
             '990049233620205171': defaultdict(str,
                         {'952_1': '$$aIn copyright; Contract$$bProtected during author lifetime plus 70 years$$cNoam Solan by Ruben Wengiel 20220214$$dייצוא דוח נתונים'}),
             '990049233630205171': defaultdict(str,
                         {'952_1': '$$aIn copyright; Contract$$bProtected during author lifetime plus 70 years$$cNoam Solan by Ruben Wengiel 20220214$$dייצוא דוח נתונים'}),
             '990049233640205

In [24]:
dd  = create_field_dict(file_path, "952")
text = dd["990049233660205171"]["952_1"]

In [26]:
dd.keys

-1

In [8]:
test_dict = defaultdict(lambda: defaultdict(str))
for record in records: 
#     print(record["001"].value())
#     test_dict[record["001"].value()] = dict
    index = 1
    for field in record.get_fields("915"):
        key = f'{field.tag}{"".join(field.indicators)}_{index}'
        key = "".join([x for x in key if x !=" "])
        value = ""
        for subfield in field.subfields:
            if len(subfield) == 1:
                value += f'$${subfield}'
            else:
                value += subfield
#         print(value)
        test_dict[record["001"].value()][key] = value
        index +=1
                
dict(test_dict)

NameError: name 'defaultdict' is not defined

In [ ]:
record_test = records[1]
for x in record_test.get_fields("700")[0].subfields:
    print(x)

In [ ]:
def create_907_value(dict_907):
    words = []
    for tag, value in dict_907.items():

        if len(value) == 0:
            return ""
        else:
            if "_" in tag:

                words.append(tag[5:] + value)
            else:
                words.append(tag[3:] + value)
    return_val =";".join(words)
    return_val.replace("$$$$", "$$")
    return return_val

In [ ]:
create_907_value(test_dict["990052543680205171"]).split(";")

In [ ]:
test_data = pd.Series(['לא ידוע [מו"ל];לא ידוע [מעצב גרפי];לא ידוע [מחבר]'], index =['יוצרים'])


In [ ]:
test_data

In [9]:
from VC_collections.marc import find_unknown_multiple_in_column

In [10]:
find_unknown_multiple_in_column(test_data, "יוצרים")

NameError: name 'test_data' is not defined

In [ ]:
test_data_unknown_and_multiple = pd.Series(['לא ידוע [מו"ל];לא ידוע [מעצב גרפי];ריבוי [מחבר]'], index =['יוצרים'])
(multiple_creators, unknown_roles, new_creators) = find_unknown_multiple_in_column(test_data_unknown_and_multiple, "יוצרים")
print(f'multiple_creators: {multiple_creators}')
print(f'unknown_roles: {unknown_roles}')
print(f'new_creators: {new_creators}')


In [ ]:
len(new_creators)

In [ ]:
test_data_unknown_and_multiple_and_known = pd.Series(['ישראל ישראל [מו"ל];לא ידוע [מעצב גרפי];ריבוי [מחבר]'], index =['יוצרים'])
(multiple_creators, unknown_roles, new_creators) = find_unknown_multiple_in_column(test_data_unknown_and_multiple_and_known, "יוצרים")
print(f'multiple_creators: {multiple_creators}')
print(f'unknown_roles: {unknown_roles}')
print(f'new_creators: {new_creators}')


In [ ]:
test_data_nounknown_and_nomultiple = pd.Series(['ישראל ישראל [מו"ל];לא ידוע [מעצב גרפי]; [מחבר]'], index =['יוצרים'])
(multiple_creators, unknown_roles, new_creators) = find_unknown_multiple_in_column(test_data_unknown_and_multiple_and_known, "יוצרים")
print(f'multiple_creators: {multiple_creators}')
print(f'unknown_roles: {unknown_roles}')
print(f'new_creators: {new_creators}')


In [ ]:
multiple_creators

In [ ]:
unknown_roles

In [ ]:
new_creators